# Antenna Pattern Analysis for 141.7001 Hz Signal

**Purpose**: Validate whether the observed 141.7001 Hz signal is consistent with
an astrophysical source by comparing predicted antenna response patterns with
observed amplitude ratios across detectors.

**Author**: José Manuel Mota Burruezo (JMMB Ψ✧)  
**Date**: October 2025  
**License**: MIT

## Theory

For a gravitational wave from sky position (RA, Dec) with polarization angle ψ,
each detector responds with amplitude:

$$h(t) = F_+(t; \text{RA, Dec}, \psi) h_+ + F_\times(t; \text{RA, Dec}, \psi) h_\times$$

where $F_+$ and $F_\times$ are the antenna pattern functions.

For a given event:
1. Extract published RA, Dec from GWOSC
2. Compute $F_+$ and $F_\times$ for each detector at coalescence time
3. Predict amplitude ratio between detectors
4. Compare with observed SNR ratio at 141.7001 Hz

**Consistency check**: If the signal is astrophysical, observed ratios should
match predicted patterns within statistical uncertainties.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy import units as u
import json

# GW-specific imports (optional, fallback to manual calculation)
try:
    from gwpy.detector import Channel
    from lal import GreenwichMeanSiderealTime
    from lal.lal import GreenwichMeanSiderealTime as GMST
except ImportError:
    print("Warning: Some packages not available, using simplified antenna patterns")

## Detector Locations and Orientations

Standard LIGO/Virgo detector parameters:

In [ ]:
# Detector locations (latitude, longitude, altitude)
DETECTORS = {
    'H1': {
        'name': 'LIGO Hanford',
        'location': EarthLocation(lat=46.4551*u.deg, lon=-119.4077*u.deg, height=142.554*u.m),
        'arm_azimuth': [180 - 36, 90 - 36],  # degrees, X and Y arms from North
    },
    'L1': {
        'name': 'LIGO Livingston',
        'location': EarthLocation(lat=30.5629*u.deg, lon=-90.7742*u.deg, height=-6.574*u.m),
        'arm_azimuth': [180 - 18, 90 - 18],  # degrees
    },
    'V1': {
        'name': 'Virgo',
        'location': EarthLocation(lat=43.6314*u.deg, lon=10.5045*u.deg, height=51.884*u.m),
        'arm_azimuth': [180 + 19.4, 90 + 19.4],  # degrees
    }
}

print("Detector configurations loaded:")
for det, info in DETECTORS.items():
    print(f"  {det}: {info['name']}")
    print(f"       Lat: {info['location'].lat:.4f}°, Lon: {info['location'].lon:.4f}°")

## Antenna Pattern Functions

Simplified calculation (exact requires full detector tensor):

In [ ]:
def compute_antenna_patterns(detector_name, ra_deg, dec_deg, gps_time, psi_deg=0):
    """
    Compute antenna pattern functions F+ and Fx for a detector.
    
    Parameters
    ----------
    detector_name : str
        Detector identifier ('H1', 'L1', 'V1')
    ra_deg : float
        Right ascension in degrees
    dec_deg : float
        Declination in degrees
    gps_time : float
        GPS time of observation
    psi_deg : float, optional
        Polarization angle in degrees (default: 0)
    
    Returns
    -------
    F_plus : float
        + polarization antenna pattern
    F_cross : float
        x polarization antenna pattern
    """
    # This is a simplified placeholder
    # For production, use lalsimulation.SimDetectorStrainREAL8TimeSeries or similar
    
    detector = DETECTORS[detector_name]
    
    # Convert to radians
    ra = np.deg2rad(ra_deg)
    dec = np.deg2rad(dec_deg)
    psi = np.deg2rad(psi_deg)
    
    # Detector latitude and longitude
    lat = detector['location'].lat.rad
    lon = detector['location'].lon.rad
    
    # Hour angle (simplified - should use GMST)
    # For now, use approximate value
    t_day = (gps_time % 86400) / 86400  # Fraction of day
    gmst = 2 * np.pi * t_day  # Simplified GMST
    hour_angle = gmst + lon - ra
    
    # Direction cosines in detector frame
    # (This is highly simplified - production should use exact detector tensor)
    theta = np.pi/2 - dec
    phi = hour_angle
    
    # Simplified antenna patterns (order of magnitude correct)
    F_plus = 0.5 * (1 + np.cos(theta)**2) * np.cos(2*phi) * np.cos(2*psi) \
             - np.cos(theta) * np.sin(2*phi) * np.sin(2*psi)
    
    F_cross = 0.5 * (1 + np.cos(theta)**2) * np.cos(2*phi) * np.sin(2*psi) \
              + np.cos(theta) * np.sin(2*phi) * np.cos(2*psi)
    
    return F_plus, F_cross


# Test with GW150914 parameters
gw150914_ra = 48.0  # degrees (approximate)
gw150914_dec = -32.0  # degrees (approximate)
gw150914_gps = 1126259462.4  # GPS time

print("\nExample: GW150914 antenna patterns")
for det in ['H1', 'L1']:
    Fp, Fc = compute_antenna_patterns(det, gw150914_ra, gw150914_dec, gw150914_gps)
    print(f"{det}: F+ = {Fp:.3f}, Fx = {Fc:.3f}")

## Event Analysis Template

For each event in GWTC-1:

In [ ]:
# Event parameters (from GWOSC)
EVENTS = {
    'GW150914': {'gps': 1126259462.4, 'ra': 48.0, 'dec': -32.0},
    'GW151226': {'gps': 1135136350.6, 'ra': 325.0, 'dec': 12.0},
    # Add more events as needed
}

# Observed SNRs at 141.7001 Hz (from multi_event_analysis.py)
OBSERVED_SNRS = {
    'GW150914': {'H1': 8.2, 'L1': 7.5},
    'GW151226': {'H1': 5.8, 'L1': 6.5},
    # Add more events as needed
}


def analyze_event_antenna_consistency(event_name, psi_deg=0):
    """
    Check antenna pattern consistency for a single event.
    """
    if event_name not in EVENTS or event_name not in OBSERVED_SNRS:
        print(f"Warning: {event_name} not in database")
        return None
    
    event = EVENTS[event_name]
    snrs = OBSERVED_SNRS[event_name]
    
    # Compute antenna patterns
    patterns = {}
    for det in snrs.keys():
        Fp, Fc = compute_antenna_patterns(det, event['ra'], event['dec'], 
                                          event['gps'], psi_deg)
        # Combined response (assuming unpolarized or averaging over psi)
        F_combined = np.sqrt(Fp**2 + Fc**2)
        patterns[det] = {'F+': Fp, 'Fx': Fc, 'F_combined': F_combined}
    
    # Predict amplitude ratio (assuming same intrinsic amplitude)
    det_list = list(snrs.keys())
    if len(det_list) >= 2:
        det1, det2 = det_list[0], det_list[1]
        predicted_ratio = patterns[det1]['F_combined'] / patterns[det2]['F_combined']
        observed_ratio = snrs[det1] / snrs[det2]
        
        return {
            'event': event_name,
            'patterns': patterns,
            'predicted_ratio': f"{det1}/{det2} = {predicted_ratio:.3f}",
            'observed_ratio': f"{det1}/{det2} = {observed_ratio:.3f}",
            'consistency': abs(predicted_ratio - observed_ratio) / observed_ratio < 0.5
        }
    
    return {'event': event_name, 'patterns': patterns}


# Analyze available events
print("\n" + "="*60)
print("ANTENNA PATTERN CONSISTENCY CHECK")
print("="*60)

results = {}
for event_name in EVENTS.keys():
    result = analyze_event_antenna_consistency(event_name)
    if result:
        results[event_name] = result
        print(f"\n{event_name}:")
        if 'predicted_ratio' in result:
            print(f"  Predicted: {result['predicted_ratio']}")
            print(f"  Observed:  {result['observed_ratio']}")
            print(f"  Consistent: {result['consistency']}")

print("\n" + "="*60)

## Visualization

In [ ]:
# Placeholder for visualization
# TODO: Add sky map with antenna response patterns
# TODO: Add predicted vs observed amplitude comparison plot

print("Visualizations: TBD")
print("  - Sky map with F+ and Fx patterns")
print("  - Predicted vs observed amplitude ratios")
print("  - Consistency score across all events")

## Export Results

In [ ]:
# Save results
output_file = '../results/antenna_pattern_check.json'

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2, default=str)

print(f"Results saved to {output_file}")

## Notes

**Limitations of this skeleton**:
1. Antenna pattern calculation is simplified (should use exact detector tensors)
2. GMST calculation is approximate (should use LAL or astropy for precision)
3. Polarization angle ψ is fixed (should marginalize or use posterior samples)
4. Statistical uncertainties not yet propagated

**Next steps**:
1. Integrate with LALSuite for exact antenna patterns
2. Use GWOSC posterior samples for RA, Dec, ψ marginalization
3. Add statistical tests (χ² test for consistency)
4. Visualize on Mollweide projection with detector response overlay